<a href="https://colab.research.google.com/github/snupet/Twittorials/blob/master/Redirect_Chain_ShortCircuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[1] Installing necessary libraries

In [39]:
%%capture
!pip install --upgrade -q gspread
!pip install requests-html

[2] Authenticating user. You will be prompted with a link to get a key. Please enter the key in the box that appears below.

In [40]:
import gspread
from google.colab import auth
auth.authenticate_user()
from requests_html import HTMLSession
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

[3] Make a copy of this template sheet: https://docs.google.com/spreadsheets/d/1pcXjJ2dpG6yxoWbQgFZbNm_dkua4n6rrno-K5k6LYSc/edit?usp=sharing. Enter in your parameters on the side.

In [45]:
#Replace with your spreadsheet name
spreadsheetName = "Template Sheet - Redirects" #@param {type:"string"}

sheet = gc.open(spreadsheetName).sheet1

#Input the starting row:
startRow =  2 #@param {type:"number"}

#Input the ending row:
endRow =  4 #@param {type:"number"}

#Input the max number of redirects
maxRedirects = 10 #@param {type: "number"}

[4] Loops through each URL in the spreadsheet and traces the redirects. Updates the spreadsheet with the page the URL redirects to, as well as the number of hops.

In [ ]:
for i in range(int(startRow), int(endRow) + 1):
  
  session = HTMLSession()
  url = sheet.cell(i, 1).value
  r = session.get(url)

  print("Current URL: " + url)
  
  numHops = len(r.history)

  if numHops != 0:
    statusCode = r.history[len(r.history)-1].status_code
  
    if maxRedirects >= 0:
      for j in range(maxRedirects - 1, len(r.history)-1):
        r.history.remove(r.history[j])
    else:
      print("Max redirects must be greater than 0")

    print("Redirects to: " +  r.url)
    print("Status code: " + str(statusCode))
    print("Amount of hops: " + str(numHops))

    sheet.update_cell(i, 2, r.url)
    sheet.update_cell(i, 3, numHops)
    sheet.update_cell(i, 4, statusCode)

    print("Intermittent URLs: ")
    for rs in r.history:
      print(rs.url)

    #Prepares sheet to update intermittent URLs by removing the first from the history, then updating the column with that first URL
    r.history.remove(r.history[0])
    sheet.update_cell(i, 5, sheet.cell(i, 1).value)

    for rs in r.history:
      sheet.update_cell(i, 5, sheet.cell(i, 5).value + ", " + str(rs.url))
  
    print("")
  else:
    print("No history found")
    sheet.update_cell(i, 2, sheet.cell(i, 1).value)
    sheet.update_cell(i, 3, numHops)
    sheet.update_cell(i, 4, r.status_code)
    sheet.update_cell(i, 5, "None")
